In [13]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"


Running local


In [14]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Jaguas_DataLoader_rainless import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [81]:
class df_generator():
    def __init__(self, root_clusters, root_audios, root_recorders, n_clusters_index = 0):
        self.n_clusters_index = n_clusters_index
        self.cluster_names = os.listdir(f"{root_clusters}")
        self.torch_clusters = torch.load(f"{root_clusters}/{self.cluster_names[n_clusters_index]}")
        self.dataframe_clusters = pd.DataFrame(self.torch_clusters)
        self.dataframe_clusters = self.dataframe_clusters.transpose()
        self.dataframe_audios = pd.read_csv(f"{root_audios}", index_col=0)
        self.dataframe_recorders = pd.read_csv(f"{root_recorders}",sep = ";", index_col = "Recorder")
        self.df_clusters_len = len(self.dataframe_clusters)
        self.df_recorders_len = len(self.dataframe_recorders)
        self.n_clusters = np.arange(len(self.torch_clusters))

    def show_clusters(self):
        print(self.cluster_names)

    def init_clusters(self):
        for cluster in self.n_clusters:
            self.dataframe_recorders[f"Cluster {cluster}"] = 0
    
    def create_clusters(self, count_cluster=False):
        self.init_clusters()
        for cluster in self.n_clusters:
            for i in range(self.df_clusters_len):  
                if self.dataframe_clusters.iloc[i][cluster] != None:
                    self.dataframe_recorders.loc[self.dataframe_clusters.iloc[i][cluster].split("_")[0], f"Cluster {cluster}"] +=1
                else:
                    pass
        if count_cluster == True:
            self.count_cluster_data()
        else:
            pass
        
        return self.dataframe_recorders
    
    def count_cluster_data(self):
        self.dataframe_recorders["Total_Clustering_Audios"] = 0
        for recorder in range(self.df_recorders_len):
            clusters = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
            clusters.append("Total_Clustering_Audios")
            self.dataframe_recorders.iloc[recorder, -1] = self.dataframe_recorders[clusters].iloc[recorder].sum()
    
    def GLM_dataframe(self):
        GLM = self.dataframe_recorders.copy()
        columns = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
        columns.append("Total_Clustering_Audios")
        GLM = GLM[columns]
        for cluster in range(len(columns[0:-1])):
            for recorder in range(self.df_recorders_len):
                GLM.iloc[recorder, cluster] = GLM.iloc[recorder, cluster]/GLM.iloc[recorder, -1]
        return GLM
    
    def regions(self):
        self.dataframe_audios["Region"] = 0
        for audio in range(len(prueba)):   
            self.dataframe_audios.loc[audio,"Region"] = self.dataframe_recorders.loc[self.dataframe_audios.loc[audio, "Recorder"]].loc["Region"]
        return self.dataframe_audios
    
    def save(self, root_save=None):
        if root_save != None:
            os.makedirs(f"{self.root_save}/dataframes", exist_ok=True)
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}_recorders.csv")
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}_audios.csv") 
            
        else:
            self.dataframe_recorders.to_csv(f"{self.cluster_names[sel.n_clusters_index]}_recorders.csv")
            self.dataframe_audios.to_csv(f"{self.cluster_names[sel.n_clusters_index]}_audios.csv")            

In [82]:
root_audios = f"{root}/Jaguas/Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv"
root_recorders = f"{root}/Jaguas/Complementary_Files/df_grabadoras_reg.csv"
root_clusters = f"{root}/Jaguas/temporal/clusters"
ecological_integrity = f"{root}/Jaguas/Complementary_Files/Indice_Integridad_Ecologica.xlsx"
df_EI = pd.read_excel(ecological_integrity)

In [128]:
dataframe = df_generator(root_clusters, root_audios, root_recorders, 
                         n_clusters_index=4)
dataframe.create_clusters(count_cluster=True)
GLM = dataframe.GLM_dataframe()
GLM = GLM.reset_index().set_index(np.arange(len(GLM)))
GLM["Mean"] = df_EI["Mean"]
GLM["Sum"] = df_EI["Max"]
GLM

,Recorder,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,Total_Clustering_Audios,Mean,Sum
0,G03,0.026316,0.074561,0.188596,0.134649,0.131140,0.014474,0.130702,0.114912,0.043860,0.140789,2280,0.572591,0.662041
1,G04,0.015337,0.073620,0.290798,0.130675,0.053988,0.000000,0.057055,0.079755,0.033742,0.265031,1630,0.587104,0.645013
2,G06,0.019746,0.069111,0.332863,0.103526,0.016925,0.000000,0.022003,0.133992,0.053597,0.248237,3545,0.784025,0.896008
3,G07,0.029762,0.081349,0.289683,0.132143,0.021825,0.000000,0.026587,0.195238,0.051587,0.171825,2520,0.370633,0.532298
4,G08,0.016529,0.051653,0.238430,0.123554,0.132231,0.003719,0.116116,0.121488,0.039256,0.157025,2420,0.587971,0.728520
5,G09,0.018207,0.070028,0.359104,0.097479,0.013725,0.000000,0.014006,0.056303,0.050420,0.320728,3570,0.723915,0.790780
6,G13,0.024119,0.092764,0.223748,0.143228,0.089425,0.001855,0.098701,0.101299,0.051948,0.172913,2695,0.571372,0.691484
7,G15,0.023810,0.070028,0.273109,0.170868,0.019888,0.000000,0.015406,0.311204,0.053221,0.062465,3570,0.519462,0.562418
8,G17,0.020548,0.068493,0.190411,0.112055,0.136164,0.011233,0.158356,0.127671,0.053425,0.121644,3650,0.534994,0.793996
9,G19,0.025641,0.074224,0.223752,0.137652,0.092848,0.000270,0.099865,0.181646,0.058030,0.106073,3705,0.447152,0.657790


In [121]:
regiones = dataframe.regions()

In [122]:
regiones

,Recording,Recorder,Date,Time,Filename,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Rain_occurrence,Rain_Intesity,Intensity_Category,Region
0,G03_20180511_120000,G03,2018/05/11,12:00:00,G03_20180511_120000.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,0,0,No_rain,1
1,G03_20180511_121600,G03,2018/05/11,12:16:00,G03_20180511_121600.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,0,0,No_rain,1
2,G03_20180511_123200,G03,2018/05/11,12:32:00,G03_20180511_123200.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,0,0,No_rain,1
3,G03_20180511_124800,G03,2018/05/11,12:48:00,G03_20180511_124800.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,0,0,No_rain,1
4,G03_20180511_130400,G03,2018/05/11,13:04:00,G03_20180511_130400.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,0,0,No_rain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20063,G58_20180704_070000,G58,2018/07/04,07:00:00,G58_20180704_070000.wav,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest,0,0,No_rain,12
20064,G58_20180704_064500,G58,2018/07/04,06:45:00,G58_20180704_064500.wav,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest,0,0,No_rain,12
20065,G58_20180704_071500,G58,2018/07/04,07:15:00,G58_20180704_071500.wav,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest,0,0,No_rain,12
20066,G58_20180704_073000,G58,2018/07/04,07:30:00,G58_20180704_073000.wav,San Rafael,Area proteccion embalse EPM,6.3471,-74.9696,1072,Forest,0,0,No_rain,12


In [123]:
prueba=dataframe.dataframe_audios
recorder = dataframe.dataframe_recorders
prueba["Region"] = 0
for audio in range(len(prueba)):   
    prueba.loc[audio,"Region"] = recorder.loc[prueba.loc[audio, "Recorder"]].loc["Region"]


In [129]:
X= GLM[[f"Cluster {i}" for i in range(0, 10)]]
X = X.to_numpy()
y = GLM["Sum"]
y = y.to_numpy()


# Using SK-Learn

In [130]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()
clf.fit(X,y)

PoissonRegressor()

In [131]:
clf.score(X,y)

0.005601068417487642

# Using StatsModels

In [132]:
gamma_model = sm.GLM(y, X, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
print(gamma_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   31
Model:                            GLM   Df Residuals:                       21
Model Family:                   Gamma   Df Model:                            9
Link Function:           InversePower   Scale:                        0.016733
Method:                          IRLS   Log-Likelihood:                 37.395
Date:                Fri, 14 Jul 2023   Deviance:                      0.35842
Time:                        21:48:39   Pearson chi2:                    0.351
No. Iterations:                     6   Pseudo R-squ. (CS):             0.6950
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            12.2561      7.974      1.537      0.1

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:307: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "


In [103]:
import statsmodels.api as sm
data = sm.datasets.scotland.load()
data.exog = sm.add_constant(data.exog)
gamma_model = sm.GLM(data.endog, data.exog, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
print(gamma_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    YES   No. Observations:                   32
Model:                            GLM   Df Residuals:                       24
Model Family:                   Gamma   Df Model:                            7
Link Function:           InversePower   Scale:                       0.0035843
Method:                          IRLS   Log-Likelihood:                -83.017
Date:                Fri, 14 Jul 2023   Deviance:                     0.087389
Time:                        21:43:35   Pearson chi2:                   0.0860
No. Iterations:                     6   Pseudo R-squ. (CS):             0.9800
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0178      0

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:307: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "


# SK-Learn example

In [ ]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()
X = [[1, 2], [2, 3], [3, 4], [4, 3]]
y = [12, 17, 22, 21]
clf.fit(X, y)
clf.score(X, y)
